In [32]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')

def rem_en(input_txt):
    words = input_txt.lower().split()
    noise_free_words = [word for word in words if word not in stop] 
    noise_free_text = " ".join(noise_free_words)
    return noise_free_text

stop = set(stopwords.words('english'))
tokeniser = RegexpTokenizer(r"\w+")
lemmatiser = WordNetLemmatizer()


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\crazy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\crazy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
df = pd.read_csv("processed_amazon_reviews.csv")

In [34]:
df = df[:100]
#df.head()
## remove html (eg </br>)
df["clean_headline"] = df["review_headline"].apply(lambda s: ' '.join(re.sub("[.,!?:;-='...@#_]", " ", s).split()))
df["clean_body"] = df["review_body"].apply(lambda s: ' '.join(re.sub("[.,!?:;-='...@#_]", " ", s).split()))

# was supposed to remove numbers?
df["clean_headline"].replace('\d+', '', regex=True, inplace=True)
df["clean_body"].replace('\d+', '', regex=True, inplace=True)

df["clean_headline"] = df["clean_headline"].apply(lambda s: rem_en(s))
df["clean_body"] = df["clean_body"].apply(lambda s: rem_en(s))

df["clean_headline"] = df["clean_headline"].apply(lambda x: tokeniser.tokenize(x))
df["clean_body"] = df["clean_body"].apply(lambda x: tokeniser.tokenize(x))

df["clean_headline"] = df["clean_headline"].apply(lambda tokens: [lemmatiser.lemmatize(token, pos='v') for token in tokens])
df["clean_body"] = df["clean_body"].apply(lambda tokens: [lemmatiser.lemmatize(token, pos='v') for token in tokens])

df[["review_body","clean_body"]]

,review_body,clean_body
0,"This cheese is so good, just wish it didn't co...","[cheese, good, wish, cost, much, worth, specia..."
1,1st class,"[st, class]"
2,Well pleased with communion bread. Descriptio...,"[well, please, communion, bread, description, ..."
3,This communion bread is soft and a bit chewy. ...,"[communion, bread, soft, bite, chewy, small, a..."
4,I'm the pastor of a church plant and the only ...,"[pastor, church, plant, thing, church, ever, c..."
...,...,...
95,These are not the original but a good mini cho...,"[original, good, mini, choice, daughter, docto..."
96,"I miss these like crazy , so I was a happy cam...","[miss, like, crazy, happy, camper, find, suppl..."
97,Tasted great!,"[taste, great]"
98,"These things are tiny, maybe the size of gummy...","[things, tiny, maybe, size, gummy, bear, norma..."


In [36]:
def calculateBOW(wordset,l_doc):
  tf_diz = dict.fromkeys(wordset,0)
  for word in l_doc:
      tf_diz[word]=l_doc.count(word)
  return tf_diz

In [55]:

full_text = []
for i in df["clean_body"]:
    full_text += i

wordset = pd.unique(full_text)
bow1 = calculateBOW(wordset,df["clean_body"][0])
bow2 = calculateBOW(wordset,df["clean_body"][3])
bow3 = calculateBOW(wordset,df["clean_body"][2])
df_bow = pd.DataFrame([bow1,bow2,bow3])
df_bow.head()

,cheese,good,wish,cost,much,worth,special,treat,br,be,...,trip,tardis,tom,baker,elsewhere,bigger,yummy,originally,shop,overseas
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
